## Step1. lora微调模型

**本示例采用qwen2-7B-instruct作为base模型，利用peft框架来进行lora微调。
主要流程为：①导入模型、指令微调数据集；②编写处理上下文的帮手函数；③设置lora参数；④训练**

对于解析pdf，我觉得可以使用pymupdf

### 下载环境


In [2]:
#下载环境
!pip install transformers>=4.37.0
!pip install torch
# !pip install PyPDF2
!pip install tqdm
!pip install accelerate

Looking in indexes: https://mirrors.ivolces.com/pypi/simple/
Looking in indexes: https://mirrors.ivolces.com/pypi/simple/
Looking in indexes: https://mirrors.ivolces.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 37.9 MB/s eta 0:00:00


In [8]:
!pip install pymupdf


Looking in indexes: https://mirrors.ivolces.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 116.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 137.5 MB/s eta 0:00:0000:0100:01


In [9]:
import os
import re
import sys
import json
import warnings
# import PyPDF2
import fitz
from tqdm import tqdm
from transformers import AutoModelForCausalLM,AutoTokenizer,Trainer,TrainingArguments
from torch.utils.data import Dataset
import copy
import torch

### 定义训练集、模型、上下文路径


In [10]:
#导入你自己构建的微调数据集
train_input_path='/root/code/LLM/exampleData_v6/finetune_exmaple.jsonl'

#导入可能用到的上下文目录路径
PDF_PATH='/root/code/LLM/exampleData_v6/pdfs'

device="cuda"

#导入base模型
model_path="/vepfs/fs_users/lkn/huggingface/hub"  #qwen2-7B-Instruct

### 定义帮手函数（用于处理上下文，解析pdf）

In [11]:
#  从PDF文件中提取文本，并以字符串列表的形式返回。
#    参数：
#        pdf_path: PDF文件的路径。
#        add_page_num: 是否在每页文本的开头添加页码。
#    返回：
#        texts: 一个字符串列表，其中每个字符串都是一页的文本。

def extract_text(pdf_path, ) -> list[str]:
    texts = []
    try:
        # Open the PDF file
        doc = fitz.open(pdf_path)
        for page_num in range(doc.page_count):
            page = doc.load_page(page_num)
            text = page.get_text("text")  # Extract text from the page
            if text:
                text = f"Page {page_num + 1}:\n{text}\n"
                texts.append(text)
    except Exception as e:
        print(f"Error while processing PDF: {e}")
    return texts


#读取doi字段，根据路径去解析相应的pdf，并根据"pages"字段来截取需要的上下文，把上下文作为user prompt append进原来的input list
def parse_pdf_and_concate(obj):
    pdf_path=obj["doi"]
    pdf_path = pdf_path.replace('/', '_').replace(' (Supporting Information)', '_si')
    pdf_path=PDF_PATH+pdf_path+'.pdf'
    attach_content_list=extract_text(pdf_path=pdf_path)
    if "pages" in obj and obj["pages"] != [1,-1] :
        #例如 pages=[5,6] 代表attach_content_list 中第4个str和第五个str
        index=obj["pages"]
        attach_content_list=attach_content_list[index[0]-1:index[1]]
    
    attached_file_content = "\nThe file is as follows:\n\n" + "".join(attach_content_list)
    #  选择性的获取上下文信息，而不是全部内容
    attached_file_content = attached_file_content[:1024]   
    obj["input"].append({"role":"user","content":attached_file_content})


### 导入模型和tokenzier

In [5]:
#导入模型
local_cache_dir = "/vepfs/fs_users/lkn/huggingface/hub"

# 加载模型和分词器时指定 cache_dir
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-7B-Instruct", cache_dir=local_cache_dir)
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-7B-Instruct", cache_dir=local_cache_dir)

Loading checkpoint shards: 100%|██████████| 4/4 [00:32<00:00,  8.18s/it]


Error: INVALID mime type: status. Must be in the format "type/subtype[;optionalparameter]"

In [9]:
#工具函数，读取测试集jsonl到list
def read_jsonl(file_path):    
    data=[]
    with open(file_path,'r',encoding='utf-8') as f:
        for line in f:
            try:
                obj=json.loads(line.strip())
                data.append(obj)
            except json.JSONDecodeError as e :
                print(f"Error decoding JSON:{e}")
    return data

train_data=read_jsonl(train_input_path)

#帮手函数，提取ideal的值，因为ideal有可能是个list，也可能是个str
def extract_ideal_value(ideal):
    if isinstance(ideal, list) and len(ideal) > 0:
        return ideal[0]
    elif isinstance(ideal, str):
        return ideal
    else:
        return None  

Error: INVALID mime type: status. Must be in the format "type/subtype[;optionalparameter]"

### 构建自定义的Dataset和Collator

In [10]:
#导入训练集
class sftDataset(Dataset):
    def __init__(self,data,tokenizer):
        #data是个list
        self.data=data
        self.tokenizer=tokenizer

    def __getitem__(self, index) :
        #把对应论文的内容concat到user content的后面。具体对pdf怎么处理由选手决定，此处仅是一个简单的示例。
        if "pages" in self.data[index]:
            #处理pdf并拼接
            parse_pdf_and_concate(self.data[index])
        input=self.data[index]["input"]
        ideal=extract_ideal_value(self.data[index]["ideal"])
        output=ideal

        input.append({"role":"assistant","content":output})
        #msg是个list：
        '''  msg= [
          {"role": "system", "content": "You are an expert in the electrolytes field. Please answer the following multiple choice question correctly.\nOnly write the option (e.g., a), b), c), or d)) without explanation."},
          {"role": "user", "content": "In the upper paper, what are the minimum and maximum intramolecular distancesnm) of dimethyl carbonate?\n\na) 0.41/0.87\nb) 0.49/0.67\nc) 0.25/0.25\nd) 0.25/0.38"},
          {"role":"assistant","content":"a) 0.41/0.87"},
            ]
        '''
        response=self.tokenizer.apply_chat_template(input,tokenize=False, add_generation_prompt=False)
        input=response.split("<|im_start|>assistant\n")[0]
        input+="<|im_start|>assistant\n"
        return dict(input_ids=input, labels=response)
    
    def __len__(self):
        return len(self.data)

#把输入从字符串改为特定的token list
class Collator(object):
    def __init__(self,only_train_response,tokenizer):
        self.only_train_response=only_train_response
        self.tokenizer=tokenizer
        if self.tokenizer.pad_token_id is None:
            self.tokenizer.pad_token_id = self.tokenizer.unk_token_id
        
    def __call__(self,batch):
        input_texts=[d["input_ids"] for d in batch]
        full_texts=[d["labels"] for d in batch]

        inputs=self.tokenizer(
            text=full_texts,
            text_target=input_texts,
            return_tensors="pt",
            padding="longest",
            max_length=1024,
            truncation=True,
            return_attention_mask=True,
        )
        labels=copy.deepcopy(inputs["input_ids"])
        if self.only_train_response:
            # ignore padding
            labels[labels==self.tokenizer.pad_token_id]=-100
            # ignore input text
            labels[torch.where(inputs["labels"] !=self.tokenizer.pad_token_id)]=-100

        inputs["labels"]=labels
        return inputs

Error: INVALID mime type: status. Must be in the format "type/subtype[;optionalparameter]"

In [11]:
sft_dataset=sftDataset(train_data,tokenizer)
only_train_response=True
data_collator=Collator(only_train_response,tokenizer)

Error: INVALID mime type: status. Must be in the format "type/subtype[;optionalparameter]"

In [12]:
!pip install peft

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 4.6 MB/s eta 0:00:00a 0:00:01


Error: INVALID mime type: status. Must be in the format "type/subtype[;optionalparameter]"

In [13]:
from peft import LoraConfig,TaskType,get_peft_model,PeftModel

Error: INVALID mime type: status. Must be in the format "type/subtype[;optionalparameter]"

### 定义lora参数

In [14]:
#定义训练参数
args = TrainingArguments(
    output_dir="./personal/Qwen2_instruct_lora",#设置输出路径
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    logging_steps=10,
    num_train_epochs=3,
    save_steps=500,
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=False
)

#定义lora参数
from peft import LoraConfig,TaskType,get_peft_model,PeftModel
config=LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "k_proj", "v_proj"],
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)


Detected kernel version 4.19.24, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss


TrainOutput(global_step=216, training_loss=0.15126380000125478, metrics={'train_runtime': 1231.0533, 'train_samples_per_second': 0.175, 'train_steps_per_second': 0.175, 'total_flos': 5941446213384192.0, 'train_loss': 0.15126380000125478, 'epoch': 3.0})

Error: INVALID mime type: status. Must be in the format "type/subtype[;optionalparameter]"

### 训练

In [ ]:
lora_model=get_peft_model(model,config)

#定义trainer
trainer=Trainer(
    model=lora_model,
    args=args,
    data_collator=data_collator,
    train_dataset=sft_dataset,
)

trainer.train()

### 【补充】平台使用帮助
baseline中模型的输出结果、微调/合并示例中模型权重的保存位置都可以根据选手的需要进行修改，可以考虑保存到个人的文件夹下（/personal/），然后再挂载到创建的数据集中。
平台数据集的使用规则可以参考：
https://bohrium-doc.dp.tech/docs/userguide/Dataset/